In [ ]:
# def make_unique(columns):
#     seen = set()
#     for item in columns:
#         fudge = 1
#         newitem = item
#         while newitem in seen:
#             fudge += 1
#             newitem = "{}_{}".format(item, fudge)
#         yield newitem
#         seen.add(newitem)

In [ ]:
# def process_df(df, name):
#     print(f"\nProcessing {name} dataset:")
#     print(f"Initial shape: {df.shape}")
    
#     # Group by date and category
#     df_grouped = df.groupby(['purchase_date', 'category']).agg({
#         'purchase_qty': 'sum',
#         'sales_price': 'sum'
#     }).reset_index()
    
#     # Pivot the data to create separate columns for each category
#     df_pivot = df_grouped.pivot(index='purchase_date', columns='category', 
#                                 values=['purchase_qty', 'sales_price'])
    
#     # Flatten and clean column names
#     df_pivot.columns = [f"{col[0]}_{clean_column_name(col[1])}" for col in df_pivot.columns]
    
#     # Sort index
#     df_pivot = df_pivot.sort_index()
    
#     # Handle NaN and inf values
#     df_pivot = df_pivot.replace([np.inf, -np.inf], np.finfo(np.float64).max)
#     df_pivot = df_pivot.interpolate()
#     df_pivot = df_pivot.dropna()
    
#     print(f"Final shape after processing: {df_pivot.shape}")
#     print(f"Columns: {df_pivot.columns.tolist()}")
    
#     return df_pivot

In [ ]:
# Fit ARIMA models and forecast for each column
forecasts = {}
rmse_results = {}

# Use only columns that exist in both train and test datasets
common_columns = train.columns.intersection(test.columns)

for column in common_columns:
    print(f"Fitting ARIMA model for {column}")
    forecast = fit_arima_and_forecast(train[column], steps=len(test))
    forecasts[column] = forecast
    rmse = sqrt(mean_squared_error(test[column], forecast))
    rmse_results[column] = rmse


In [ ]:
# Create a DataFrame with all forecasts
forecast_df = pd.DataFrame(forecasts, index=test.index)

print("\nRMSE for each variable:")
for column, rmse in rmse_results.items():
    print(f"{column}: {rmse}")

In [ ]:
# Plot results for each category
import matplotlib.pyplot as plt

categories = df['category'].unique()
n_categories = len(categories)
fig, axes = plt.subplots(n_categories, 2, figsize=(20, 7*n_categories))

for i, category in enumerate(categories):
    qty_col = f'purchase_qty_{clean_column_name(category)}'
    price_col = f'sales_price_{clean_column_name(category)}'
    
    if qty_col in common_columns:
        axes[i, 0].plot(train.index, train[qty_col], label='Train')
        axes[i, 0].plot(test.index, test[qty_col], label='Test')
        axes[i, 0].plot(forecast_df.index, forecast_df[qty_col], color='red', label='Forecast')
        axes[i, 0].set_title(f'{category} - Purchase Quantity')
        axes[i, 0].legend()
        axes[i, 0].set_xlabel('Date')
        axes[i, 0].set_ylabel('Quantity')
    
    if price_col in common_columns:
        axes[i, 1].plot(train.index, train[price_col], label='Train')
        axes[i, 1].plot(test.index, test[price_col], label='Test')
        axes[i, 1].plot(forecast_df.index, forecast_df[price_col], color='red', label='Forecast')
        axes[i, 1].set_title(f'{category} - Sales Price')
        axes[i, 1].legend()
        axes[i, 1].set_xlabel('Date')
        axes[i, 1].set_ylabel('Price')

plt.tight_layout()
plt.show()